In [43]:
import os
import warnings
import argparse
import numpy as np
import pandas as pd
from typing import Dict, Any, List, Tuple
from collections import Counter
from scipy.optimize import minimize

from sklearn.base import BaseEstimator
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import (
    BayesianRidge, ElasticNet, Lasso, LinearRegression, Ridge, SGDRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
)
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from xgboost import XGBRegressor
from collections import Counter
from typing import Union, List, Tuple, Dict, Any
from scipy.stats import pearsonr
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer, )
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from sklearn.exceptions import NotFittedError

import pytorch_lightning as pl


from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig, ModelConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

# -----
# Import the data loading and model evaluation code you shared
# -----
import sys 
sys.path.append('..')
from src.data.process_data import load_dataset, split_dataset
from src.model.evaluation import train_evaluate_sklearn_pipeline
from src.model.evaluation import weighted_pearson

# Suppress sklearn ConvergenceWarnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [19]:
# Define args object 
args = argparse.Namespace()
args.seed = 42
args.data_path = "../data"

In [ ]:
drug_syn_path = os.path.join(args.data_path, 'drug_synergy.csv')
from collections import Counter
from typing import Union, List, Tuple, Dict, Any
from scipy.stats import pearsonr
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer, )
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from sklearn.exceptions import NotFittedError

import pytorch_lightning as pl


from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig, ModelConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

if not os.path.exists(drug_syn_path):
    print(f"Drug synergy file not found at {drug_syn_path}")
    exit(1)
if not os.path.exists(cell_lines_path):
    print(f"Cell lines file not found at {cell_lines_path}")
    exit(1)
if not os.path.exists(drug_portfolio_path):
    print(f"Drug portfolio file not found at {drug_portfolio_path}")
    exit(1)

full_dataset_df, _ = load_dataset(
            drug_syn_path,
            cell_lines_path,
            drug_portfolio_path
        )

# Split dataset into train, test, and leaderboard
datasets = split_dataset(full_dataset_df)

No SMILES fingerprints provided. Skipping the step.


In [21]:
models = [
        ('Linear Regression', LinearRegression()),
        ('Ridge Regression', Ridge()),
        ('Lasso Regression', Lasso()),
        ('ElasticNet Regression', ElasticNet()),
        ('Bayesian Ridge Regression', BayesianRidge()),
        ('Stochastic Gradient Descent', SGDRegressor(max_iter=1000, tol=1e-3, random_state=args.seed)),

        ('Decision Tree', DecisionTreeRegressor(random_state=args.seed)),
        ('Random Forest', RandomForestRegressor(random_state=args.seed)),
        ('Extra Trees', ExtraTreesRegressor(random_state=args.seed)),

        ('AdaBoost', AdaBoostRegressor(random_state=args.seed)),
        ('Gradient Boosting', GradientBoostingRegressor(random_state=args.seed)),
        ('XGBRegressor', XGBRegressor(random_state=args.seed)),

        ('Support Vector Regression', SVR()),
        ('Gaussian Process', GaussianProcessRegressor()),
    ]

In [23]:
class EmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, categorical_features):
        self.model = model
        self.embedding_transformer = CategoricalEmbeddingTransformer(model)
        self.categorical_features = categorical_features

    def fit(self, X, y=None):
        # No fitting required
        return self

    def transform(self, X):
        # Check if the model and transformer are correctly initialized
        if not hasattr(self.embedding_transformer, '_mapping'):
            raise NotFittedError("The CategoricalEmbeddingTransformer is not fitted yet.")
        
        # Ensure that X contains the required categorical features
        missing_cols = set(self.categorical_features) - set(X.columns)
        if missing_cols:
            raise ValueError(f"Missing categorical columns in input data: {missing_cols}")

        # Transform the categorical features into embeddings
        X_transformed = self.embedding_transformer.transform(X[self.categorical_features])
        # Extract the embedding columns
        embedding_cols = [col for col in X_transformed.columns if '_embed_dim_' in col]
        # Return the embeddings as a numpy array
        return X_transformed[embedding_cols].values

In [25]:
# Extract training data
X_train = datasets['train']['X']
y_train = datasets['train']['y']
train_comb_id = datasets['train']['comb_id']


categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
continuous_features = X_train.select_dtypes(include=['number']).columns.tolist()


print("Categorical features:", categorical_features)
print("Continuous features:", continuous_features)

# Preprocessing pipelines for numeric and categorical features
numeric_pipeline = Pipeline(steps=[
    ('selector', FunctionTransformer(lambda X: X[continuous_features], validate=False)),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create the EmbeddingTransformer
tabular_model = TabularModel.load_model('../models_weights/seed_42/CategoryEmbedding_model.ckpt',
                                         map_location=torch.device("cpu"))
embedding_transformer = EmbeddingTransformer(tabular_model, categorical_features)

embedding_pipeline = Pipeline(steps=[
    ('selector', FunctionTransformer(lambda X: X[categorical_features], validate=False)),
    ('embedding_transformer', embedding_transformer)
    ])

preprocessor = FeatureUnion(transformer_list=[
    ('num', numeric_pipeline),
    ('embedding', embedding_pipeline)
    ])

Categorical features: ['Cell line name', 'Compound A', 'Compound B', 'GDSC tissue descriptor 2', 'MSI', 'Growth properties', 'Putative target_A', 'Function_A', 'Pathway_A', 'Putative target_B', 'Function_B', 'Pathway_B']
Continuous features: ['Max. conc. A', 'IC50 A', 'H A', 'Einf A', 'Max. conc. B', 'IC50 B', 'H B', 'Einf B']


2025-03-13 15:25:41,235 - {pytorch_tabular.tabular_model:167} - INFO - Experiment Tracking is turned off

2025-03-13 15:25:41,242 - {pytorch_tabular.tabular_model:342} - INFO - Preparing the Trainer

In [31]:
model = LinearRegression()
model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
        ])

# Train and evaluate the model
# Train the model on the training data
model_pipeline.fit(X_train, y_train)

X_lb = datasets['lb']['X']
y_lb = datasets['lb']['y']
lb_comb_id = datasets['lb']['comb_id']

# Evaluate the model on the leaderboard data
lb_preds = model_pipeline.predict(X_lb)
weighted_pear, pear_weights_df = weighted_pearson(lb_comb_id, y_lb, lb_preds)


Output()

Output()

We train each model and make predictions on the LB set

In [33]:
results_dict = {}
predictions_dict = {}
# store the prediction for all models in models
for model_name, model in models:
    print(f"Training and evaluating {model_name}...")
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
        ])

    # Train and evaluate the model
    # Train the model on the training data
    model_pipeline.fit(X_train, y_train)

    X_lb = datasets['lb']['X']
    y_lb = datasets['lb']['y']
    lb_comb_id = datasets['lb']['comb_id']

    # Evaluate the model on the leaderboard data
    lb_preds = model_pipeline.predict(X_lb)
    weighted_pear, pear_weights_df = weighted_pearson(lb_comb_id, y_lb, lb_preds)

    results_dict[model_name] = weighted_pear
    predictions_dict[model_name] = lb_preds
    

Output()

Training and evaluating Linear Regression...


Output()

Output()

Training and evaluating Ridge Regression...


...

Training and evaluating Support Vector Regression...


Output()

Output()

Training and evaluating Gaussian Process...


Output()

# Create the ensemble 
## Greedy forward uniform averaging

In [36]:
def greedy_ensemble_selection(
    predictions_dict: Dict[str, np.ndarray],
    y_true: np.ndarray,
    comb_id: np.ndarray,
) -> Tuple[List[str], List[float], List[float]]:
    """
    Perform greedy forward selection to build an ensemble that maximizes
    Weighted Pearson Correlation (WPC) on the LB set.

    Parameters
    ----------
    predictions_dict : dict
        Dictionary of model name -> prediction array.
    y_true : np.ndarray
        Ground truth LB synergy scores.
    comb_id : np.ndarray
        LB combination IDs.

    Returns
    -------
    selected_models : list of str
        Ordered list of selected model names in the ensemble.
    performance_trace : list of float
        WPC of the ensemble after each step.
    deltas : list of float
        Performance gain added by each model.
    """
    model_names = list(predictions_dict.keys())
    pred_matrix = np.stack([predictions_dict[name] for name in model_names], axis=1)  # shape (n_samples, n_models)

    selected_indices = []
    remaining_indices = list(range(len(model_names)))
    performance_trace = []
    deltas = []

    best_wpc = -np.inf
    while remaining_indices:
        best_candidate = None
        best_candidate_wpc = best_wpc

        for idx in remaining_indices:
            trial_indices = selected_indices + [idx]
            trial_preds = pred_matrix[:, trial_indices].mean(axis=1)
            trial_wpc, _ = weighted_pearson(comb_id, trial_preds, y_true)

            if trial_wpc > best_candidate_wpc:
                best_candidate_wpc = trial_wpc
                best_candidate = idx

        if best_candidate is None or best_candidate_wpc <= best_wpc:
            break  # no further improvement

        selected_indices.append(best_candidate)
        remaining_indices.remove(best_candidate)

        delta = best_candidate_wpc - best_wpc
        deltas.append(delta)
        best_wpc = best_candidate_wpc
        performance_trace.append(best_wpc)

        print(f"Added: {model_names[best_candidate]:<25} | "
              f"WPC: {best_wpc:.5f} | ΔWPC: {delta:.5f}")

    selected_models = [model_names[i] for i in selected_indices]
    return selected_models, performance_trace, deltas

In [37]:
selected_models, performance_trace, deltas = greedy_ensemble_selection(
    predictions_dict, y_lb, lb_comb_id)

print("\nSelected models:")
print(selected_models)
print("\nPerformance trace:")
print(performance_trace)
print("\nPerformance deltas:")
print(deltas)
    

/home/manu/CODE/cat_drug_synergy/notebooks/../src/model/evaluation.py:85: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho_i, _ = pearsonr(y_pred[mask], y_true[mask])


Added: Random Forest             | WPC: 0.29009 | ΔWPC: inf
Added: Stochastic Gradient Descent | WPC: 0.30187 | ΔWPC: 0.01178
Added: Extra Trees               | WPC: 0.30747 | ΔWPC: 0.00560
Added: Support Vector Regression | WPC: 0.31049 | ΔWPC: 0.00301
Added: Bayesian Ridge Regression | WPC: 0.31279 | ΔWPC: 0.00230
Added: Gradient Boosting         | WPC: 0.31351 | ΔWPC: 0.00072
Added: Lasso Regression          | WPC: 0.31932 | ΔWPC: 0.00581
Added: Gaussian Process          | WPC: 0.31932 | ΔWPC: 0.00000

Selected models:
['Random Forest', 'Stochastic Gradient Descent', 'Extra Trees', 'Support Vector Regression', 'Bayesian Ridge Regression', 'Gradient Boosting', 'Lasso Regression', 'Gaussian Process']

Performance trace:
[0.29009165169768686, 0.30187131371474474, 0.30747459058220883, 0.31048737064910714, 0.3127886575799381, 0.3135065367056308, 0.31932147897750396, 0.3193214789777846]

Performance deltas:
[inf, 0.01177966201705788, 0.005603276867464091, 0.003012780066898313, 0.002301286

## Greedy forward + weighted averaging

In [41]:
def optimize_weights(preds: np.ndarray, y_true: np.ndarray, comb_id: np.ndarray) -> Tuple[np.ndarray, float]:
    """
    Finds optimal weights for combining model predictions to maximize WPC.
    """
    n_models = preds.shape[1]

    def loss(weights):
        weights = np.maximum(weights, 0)
        weights = weights / (weights.sum() + 1e-8)  # normalize to sum to 1
        y_ensemble = np.dot(preds, weights)
        wpc, _ = weighted_pearson(comb_id, y_ensemble, y_true)
        return -wpc  # minimize negative WPC

    initial_weights = np.ones(n_models) / n_models
    bounds = [(0.0, 1.0) for _ in range(n_models)]
    result = minimize(loss, initial_weights, method='SLSQP', bounds=bounds)

    final_weights = result.x / result.x.sum()
    best_wpc = -result.fun
    return final_weights, best_wpc


def greedy_weighted_ensemble_selection(
    predictions_dict: Dict[str, np.ndarray],
    y_true: np.ndarray,
    comb_id: np.ndarray,
) -> Tuple[List[str], List[float], List[float], List[np.ndarray]]:
    """
    Greedy forward ensemble selection with weight optimization at each step.

    Returns:
        - selected_models: list of model names
        - performance_trace: list of WPCs after each addition
        - deltas: WPC improvement per step
        - weights_trace: list of weight vectors used at each stage
    """
    model_names = list(predictions_dict.keys())
    all_preds = np.stack([predictions_dict[m] for m in model_names], axis=1)  # (n_samples, n_models)

    selected_indices = []
    remaining_indices = list(range(len(model_names)))
    performance_trace = []
    weights_trace = []
    deltas = []

    current_wpc = -np.inf

    while remaining_indices:
        best_candidate = None
        best_candidate_wpc = current_wpc
        best_candidate_weights = None

        for idx in remaining_indices:
            trial_indices = selected_indices + [idx]
            trial_preds = all_preds[:, trial_indices]

            weights, trial_wpc = optimize_weights(trial_preds, y_true, comb_id)

            if trial_wpc > best_candidate_wpc:
                best_candidate = idx
                best_candidate_wpc = trial_wpc
                best_candidate_weights = weights

        if best_candidate is None or best_candidate_wpc <= current_wpc:
            break  # no improvement

        selected_indices.append(best_candidate)
        remaining_indices.remove(best_candidate)

        delta = best_candidate_wpc - current_wpc
        deltas.append(delta)
        performance_trace.append(best_candidate_wpc)
        weights_trace.append(best_candidate_weights)

        current_wpc = best_candidate_wpc

        print(f"Added: {model_names[best_candidate]:<25} | "
              f"WPC: {current_wpc:.5f} | ΔWPC: {delta:.5f}")

    selected_models = [model_names[i] for i in selected_indices]
    return selected_models, performance_trace, deltas, weights_trace

In [44]:
# Your input data
y_lb = datasets['lb']['y'].values
lb_comb_id = datasets['lb']['comb_id'].values

# Run the algorithm
selected_models, performance_trace, deltas, weights_trace = greedy_weighted_ensemble_selection(
    predictions_dict=predictions_dict,
    y_true=y_lb,
    comb_id=lb_comb_id
)

print("\nSelected models:")
print(selected_models)
print("\nPerformance trace:")
print(performance_trace)
print("\nPerformance deltas:")
print(deltas)
print("\nWeights trace:")

/home/manu/CODE/cat_drug_synergy/notebooks/../src/model/evaluation.py:85: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rho_i, _ = pearsonr(y_pred[mask], y_true[mask])


Added: Random Forest             | WPC: 0.29009 | ΔWPC: inf
Added: ElasticNet Regression     | WPC: 0.30858 | ΔWPC: 0.01849
Added: Stochastic Gradient Descent | WPC: 0.32002 | ΔWPC: 0.01144
Added: Gradient Boosting         | WPC: 0.32652 | ΔWPC: 0.00650
Added: XGBRegressor              | WPC: 0.32686 | ΔWPC: 0.00034


KeyboardInterrupt: 

## Stacking

In [80]:
results_dict = {}
predictions_dict = {}
test_results = {}
test_predictions_dict = {}
# store the prediction for all models in models
for model_name, model in models:
    print(f"Training and evaluating {model_name}...")
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
        ])

    # Train and evaluate the model
    # Train the model on the training data
    model_pipeline.fit(X_train, y_train)

    X_lb = datasets['lb']['X']
    y_lb = datasets['lb']['y']
    lb_comb_id = datasets['lb']['comb_id']

    # Evaluate the model on the leaderboard data
    lb_preds = model_pipeline.predict(X_lb)
    weighted_pear, pear_weights_df = weighted_pearson(lb_comb_id, y_lb, lb_preds)

    results_dict[model_name] = weighted_pear
    predictions_dict[model_name] = lb_preds
    # Evaluate the model on the test data
    X_test = datasets['test']['X']
    y_test = datasets['test']['y']
    test_comb_id = datasets['test']['comb_id']

    # Evaluate the model on the leaderboard data
    test_preds = model_pipeline.predict(X_test)
    weighted_pear, pear_weights_df = weighted_pearson(test_comb_id, y_test, test_preds)

    test_results[model_name] = weighted_pear
    test_predictions_dict[model_name] = test_preds
    

print("Finished training and evaluating all models.")

Output()

Training and evaluating Linear Regression...


Output()

Output()

Output()

...

Output()

Training and evaluating Gaussian Process...


Output()

Output()

Finished training and evaluating all models.


In [102]:
model_names = list(test_predictions_dict.keys())
#model_names = ['Linear Regression',  'ElasticNet Regression',]

# Build matrix of base model predictions
test_preds_arr = np.column_stack([test_predictions_dict[m] for m in model_names])  # shape (n_samples_test, n_models)
lb_preds_arr = np.column_stack([predictions_dict[m] for m in model_names])         # shape (n_samples_lb, n_models)

# Train linear model on test predictions
linear_model = ElasticNet()
linear_model.fit(test_preds_arr, y_test)

# Predict on test set
test_preds_linear_model = linear_model.predict(test_preds_arr)
test_wpc, _ = weighted_pearson(test_comb_id, test_preds_linear_model, y_test)
print(f"Linear model on test set WPC: {test_wpc:.5f}")

# Predict on leaderboard set
lb_preds_linear_model = linear_model.predict(lb_preds_arr)
lb_wpc, _ = weighted_pearson(lb_comb_id, lb_preds_linear_model, y_lb)
print(f"Linear model on leaderboard WPC: {lb_wpc:.5f}")

Linear model on test set WPC: 0.31617
Linear model on leaderboard WPC: 0.25765


In [115]:
# Step 1: Wrap your preprocessor
preprocesing = Pipeline(steps=[('preprocessor', preprocessor)])

# Step 2: Apply preprocessing to X_test and X_lb
X_test_preprocessed = preprocesing.transform(X_test)
X_lb_preprocessed = preprocesing.transform(X_lb)

# Step 3: Stack model predictions
model_names = list(test_predictions_dict.keys())

test_model_preds = np.column_stack([test_predictions_dict[m] for m in model_names])
lb_model_preds = np.column_stack([predictions_dict[m] for m in model_names])

# Step 4: Concatenate predictions + preprocessed features
test_stack_input = np.concatenate([test_model_preds, X_test_preprocessed], axis=1)
lb_stack_input = np.concatenate([lb_model_preds, X_lb_preprocessed], axis=1)

# Step 5: Train meta-model 
meta_model = BayesianRidge()
meta_model.fit(test_stack_input, y_test)

# Step 6: Predict and evaluate
test_preds = meta_model.predict(test_stack_input)
test_wpc, _ = weighted_pearson(test_comb_id, test_preds, y_test)
print(f"Meta-model (preds + features) - test WPC: {test_wpc:.5f}")

lb_preds = meta_model.predict(lb_stack_input)
lb_wpc, _ = weighted_pearson(lb_comb_id, lb_preds, y_lb)
print(f"Meta-model (preds + features) - leaderboard WPC: {lb_wpc:.5f}")

Output()

Output()

Meta-model (preds + features) - test WPC: 0.33278
Meta-model (preds + features) - leaderboard WPC: 0.27708
